# Try rechunking xarray dataset to Zarr using Rechunker

In [1]:
import xarray as xr
from dask.distributed import Client, performance_report, LocalCluster
from rechunker import rechunk
import zarr

In [2]:
cluster = LocalCluster(n_workers=12, threads_per_worker=1)

In [3]:
cluster

In [4]:
client = Client(cluster)

In [5]:
src = 'http://thredds.ucar.edu/thredds/dodsC/grib/NCEP/HRRR/CONUS_2p5km/Best'
temp = '/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/tmp'
target = '/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/EarthMap/Projects/de_bay/foo'

ds = xr.open_dataset(src, chunks=dict(time=1))

Initialize all data_vars as unchunked, as we will only chunk 3D and 4D variables

In [6]:
# try just one variable for now
ds = ds[['Temperature_height_above_ground']]

In [7]:
ds

<xarray.Dataset>
Dimensions:                          (height_above_ground1: 1, time: 105, x: 2145, y: 1377)
Coordinates:
  * x                                (x) float32 -2763.217 ... 2681.906
  * y                                (y) float32 -263.79062 ... 3230.8406
  * time                             (time) datetime64[ns] 2020-10-25 ... 202...
    reftime                          (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
  * height_above_ground1             (height_above_ground1) float32 2.0
Data variables:
    Temperature_height_above_ground  (time, height_above_ground1, y, x) float32 dask.array<chunksize=(1, 1, 1377, 2145), meta=np.ndarray>
Attributes:
    Originating_or_generating_Center:                                        ...
    Originating_or_generating_Subcenter:                                     ...
    GRIB_table_version:                                                      ...
    Type_of_generating_process:                                              ...
    Analysis_or_forecast_generating_process_identifier_defined_by_originating...
    file_format:                                                             ...
    Conventions:                                                             ...
    history:                                                                 ...
    featureType:                                                             ...
    _CoordSysBuilder:                                                        ...

In [8]:
chunk_plan={}
for var in ds.variables:
    chunk_plan[var]=None

Specify chunks for 4D variables

In [9]:
chunks= {'time':40, 'x':300, 'y':300, 'height_above_ground1':1}
for var in ds.data_vars:
    if len(ds[var].dims)==4:
        var_chunk = {}
        for dim in ds[var].dims:
            var_chunk[dim] = chunks[dim]
        chunk_plan[var] = var_chunk

In [10]:
import shutil
try:
    shutil.rmtree(temp)
except:
    pass
try:
    shutil.rmtree(target)
except:
    pass

In [11]:
max_mem = '4GB'
array_plan = rechunk(ds, chunk_plan, max_mem, target, temp_store=temp)

AttributeError: 'Array' object has no attribute 'attrs'

In [ ]:
with performance_report(filename="dask-report.html"):
    result = array_plan.execute(retries=10)

In [ ]:
cluster.close()